In [1]:
from moviepy.audio.fx.all import *
from moviepy.editor import *
from moviepy.video.fx.all import *

from skimage.filters import gaussian

In [2]:
### Defining file paths
working_dir = '/home/rohit/Videos/sasha_clips/gi_e1_clips/'
original_video_file = working_dir + 'gi_e1.mp4'
subclip_timestamps_file = working_dir + 'subclip_timestamps.txt'
subtitle_folder = working_dir + 'subtitles/'
generated_clips_directory = working_dir + 'generated_clips/'
temp_folder = working_dir + 'temp/'

In [3]:
### Read in subclip_timestamps.txt and save data in dictionary

with open(subclip_timestamps_file) as f:
    subclip_data = f.readlines()
    
#Stripping the trailing newline character in each line
subclip_data = [line.rstrip() for line in subclip_data]

#Defining empty dictionary
clips = {}

#Looping through subclip_data
for i in range(len(subclip_data)):
    
    #Extracting a line out
    line = subclip_data[i]
    
    #If line begins with a 2 digit number
    if line[0:2].isdigit():
        
        #Defining a temporary dictionary
        #that later gets assigned to clip
        tmp = {}
        
        #Adding title to tmp
        tmp['title'] = subclip_data[i]
        
        #Extracting the start and end time strings
        t_start = subclip_data[i+1]
        t_end = subclip_data[i+2]
        
        #Split times by comma to get hour min sec
        t_start = t_start.split(',')
        t_end = t_end.split(',')
        
        #Converting timestamp to tuple
        tmp['start'] = (int(t_start[0]), int(t_start[1]), float(t_start[2]))
        tmp['end'] = (int(t_end[0]), int(t_end[1]), float(t_end[2])) 
        
        #Adding tmp to clip
        clips[int(line[0:2])] = tmp

In [4]:
#function to crop the video
def crop_clip(clip):
    
    #Extracting the size
    w, h = clip.size
    
    #Cropping the clip for portrait
    cropped_clip = crop(clip, width=h, height=h, x_center=0.5*w, y_center=0.5*h)
    
    return cropped_clip

In [5]:
#Adding logo
def add_logo(clip):
    
    #Defining logo
    logo = (ImageClip("/home/rohit/Videos/sasha_clips/logos/logo_name/logo_name_12.png")
            .set_duration(clip.duration)
            .set_position((-5, -10))
            .resize(0.20)
            .crossfadeout(0.3))
    
    ##Defining logo
    #logo = (ImageClip("/home/rohit/Videos/sasha_clips/logos/logo_symbol_only/symbol6.png")
    #        .set_duration(clip.duration)
    #        .set_position((-35, -40))
    #        .resize(0.15)
    #        .crossfadeout(0.3))
    
    ##Defining handle
    #handle = (TextClip('SASHA COBRA', font='Montserrat-SemiBold', fontsize=14, color='#423634', 
    #               align='center', kerning=5)
    #          .set_position((80, 20))
    #          .set_duration(clip.duration)
    #          .crossfadeout(0.3))
    
    ##Composition
    #clip_with_logo = CompositeVideoClip([clip, logo, handle])
    
    #Composition
    clip_with_logo = CompositeVideoClip([clip, logo])
    
    return clip_with_logo

In [6]:
#Defining a function that can take a string of srt timestamp
#and convert it to format that MoviePy needs
def timestamp_convert(srt_time):
    
    '''
    Expects a string in the form 'hours:min:seconds,milliseconds'
    Returns (hours, minutes, seconds.milliseconds)
    '''
    
    #Splitting on the :
    srt_time = srt_time.split(':')
    
    #Extracting hour, minute and seconds
    hour = int(srt_time[0])
    minute = int(srt_time[1])
    seconds = float(srt_time[2].replace(',', '.'))
    
    #Converting to tuple
    moviepy_time = (hour, minute, seconds)
    
    return moviepy_time

In [7]:
#Defining a function to generate timestamps for subclips
#Takes into account minute and seconds
def timestamp_gen(start, increment):
    """
    start - start time tuple (hours, min, seconds)
    increment - in seconds
    """
    
    #Setting stop tuple to be same as start
    stop = start
    
    #Converting to a list to increment as tuple is immutable
    temp_list = list(stop)
    
    #Adding length of post clip animation duration to the end of the main clip
    new_min = temp_list[2] + increment
    
    #Updating the list based on whether the incremented minute
    #is greater than or less than 60min as the hour has to be 
    #accounted for.
    if new_min < 60:
        temp_list[2] = new_min
    elif new_min > 60:
        temp_list[1] += 1
        temp_list[2] = new_min-60
        
    #Converting list back to tuple
    stop = tuple(temp_list)
    
    return stop

In [8]:
#Function to extract contents of subtitle file
def extract_subtitles(subtitle_file):
    
    #Extracting contents of srt file into a list
    with open(subtitle_file) as f:
        sub_content = f.readlines()
        
    #print(sub_content)
    
    #Defining a counter to keep track of the max number of 
    #subtitle sections
    max_subtitle_section = 0
    
    #Defining a dictionary to keep track of subtitle content
    #This is a dicionary that stores another dictionary
    subtitle = {}
    
    #Remove the \n at the end of each line
    sub_content = [line.rstrip() for line in sub_content]
        
    for i in range(len(sub_content)):
        
        #Check if string is number
        if sub_content[i].isdigit():
            
            #Create a temp dictionary
            temp = {}
            
            #Extract the timestamps
            timestamps = sub_content[i+1]
            
            #Split the timestamps by space
            timestamps = timestamps.split(' ')
            
            #Extracting the start and end time for the subtitle
            temp['start'] = timestamp_convert(timestamps[0])
            temp['end'] = timestamp_convert(timestamps[2])
            
            #Extract the subtitle text
            temp['text'] = sub_content[i+2]
            
            #Adding the temp dictionary to the subtitle dictionary
            subtitle[int(sub_content[i])] = temp
            
    return subtitle

In [9]:
#Defining a function to blur a frame
def blur(image):
    
    """
    Returns a blurred image using gaussian filter
    with radius
    """
    
    return gaussian(image.astype(float), sigma=20)

In [10]:
#Function to stitch subtitiles
#It generates the subclips, adds subtitles and creates a transition
def stitch_subtitles(subtitle, clip):
    
    #Defining list for concatenation
    clips_for_concatenation = []
    
    #Defining sizes
    textclip_size = (1000, 110)
    subtitle_pos = ('center', 960)
    subtitle_font_size = 32
    
    #Looping through each subtitle section
    for i in subtitle.keys():
        
        #Extracting the clip that is in between two subtitles
        #and appending to the list for concatenation
        if i != len(subtitle.keys()):
            
            #Extracting the clip
            subtitle[i]['clip'] = clip.subclip(subtitle[i]['start'], subtitle[i]['end'])
        
            #Saving the duration
            subtitle[i]['duration'] = subtitle[i]['clip'].duration
        
            #Generating the text clip
            subtitle[i]['text_clip'] = TextClip(subtitle[i]['text'], font='Ubuntu-Bold', fontsize=subtitle_font_size, 
                                       color='#071a13', size=textclip_size, method='caption', bg_color='#b6b4b2', 
                                       align='center').set_opacity(0.75)
        
            #Adding the text clip to the clip
            subtitle[i]['clip'] = CompositeVideoClip([subtitle[i]['clip'], subtitle[i]['text_clip']
                                                      .set_pos(subtitle_pos)]).set_duration(subtitle[i]['duration'])
        
            #Adding a transition clip for the bit in between 2 subtitles
            subtitle[i]['transition'] = clip.subclip(subtitle[i]['end'], subtitle[i+1]['start'])
            
            #Storing clips for concatenation
            clips_for_concatenation.append(subtitle[i]['clip'])
            clips_for_concatenation.append(subtitle[i]['transition'])
            
        #For the last clip, ensuring that it goes till the end
        else:
            
            #Extraction of clip
            subtitle[i]['clip'] = clip.subclip(subtitle[i]['start'], clip.duration)
            
            #Saving the duration
            subtitle[i]['duration'] = subtitle[i]['clip'].duration
        
            #Generating the text clip
            subtitle[i]['text_clip'] = TextClip(subtitle[i]['text'], font='Ubuntu-Bold', fontsize=subtitle_font_size, 
                                       color='#071a13', size=textclip_size, method='caption', bg_color='#b6b4b2', 
                                       align='center').set_opacity(0.75)
            
            #Adding the text clip the clip
            subtitle[i]['clip'] = subtitle[i]['clip'] = CompositeVideoClip([subtitle[i]['clip'], subtitle[i]['text_clip']
                                                      .set_pos(subtitle_pos)]).set_duration(subtitle[i]['duration'])
        
            #Storing clip for concatenation
            clips_for_concatenation.append(subtitle[i]['clip'])
            
    #Concatenating the clips
    main_clip = concatenate_videoclips(clips_for_concatenation, method='chain')
    
    return main_clip

In [11]:
#Adding subtitle
def add_subtitle(clip, title):
    
    #Defining subtitle file location
    subtitle_file = subtitle_folder + title + '.srt'
    
    #Extract the data in subtitle file
    subtitle = extract_subtitles(subtitle_file)
    
    #Stitching the subtitles
    main_clip = stitch_subtitles(subtitle, clip)
    
    return main_clip

In [12]:
#Adding post clip animation
def add_post_clip_ani(main_clip_end_time):
    
    #Defining how long we want the post clip animation to be
    #This value comes from correctly cutting the music at the 
    #beginning of the original file.
    post_clip_duration = 9.51 #seconds
    
    #Computing time steps for the complete clip
    post_clip_start = main_clip_end_time
    post_clip_end = timestamp_gen(post_clip_start, post_clip_duration)
    
    #Extracting the subclip
    post_clip = VideoFileClip(original_video_file).subclip(post_clip_start, post_clip_end)
    
    #Cropping
    post_clip_crop = crop_clip(post_clip)
    
    #Muting audio in post clip
    post_clip_mute = post_clip.volumex(0)
    
    #Defining the subclip durations of the post clip
    #We have 4 subclips based on the "eh" sound in the music
    
    #Defining the timestamps in a dictionary
    pc_t = {}
    
    pc_t['1_start'] = (0, 0, 0.0)
    pc_t['1_end'] = (0, 0, 1.111)
    pc_t['2_start'] = pc_t['1_end']
    pc_t['2_end'] = (0, 0, 3.544)
    pc_t['3_start'] = pc_t['2_end']
    pc_t['3_end'] = (0, 0, 6.040)
    pc_t['4_start'] = pc_t['3_end']
    pc_t['4_end'] = (0, 0, 8.47)
    pc_t['5_start'] = pc_t['4_end']
    pc_t['5_end'] = (0, 0, post_clip.duration)
    
    #Extracting the subclips corresponding to the subclip stamps
    #also applying blur after extraction
    #storing in a dictionary
    pc = {}
    for i in range(1, 6):
        pc[i] = post_clip_crop.subclip(pc_t[str(i)+'_start'], pc_t[str(i)+'_end']) 
        pc[i] = pc[i].fl_image(blur)
    
    #Defining text clips
    getting_intimate_1 = (TextClip('GETTING INTIMATE', font='Montserrat-SemiBold', fontsize=85, color='white',
                                align='center', kerning=10)
                          .set_position('center', 'center')
                          .set_duration(pc[1].duration)
                          .crossfadein(0.3))
    
    getting_intimate_2 = (TextClip('GETTING INTIMATE', font='Montserrat-SemiBold', fontsize=85, color='white',
                                align='center', kerning=10)
                          .set_position('center', 'center')
                          .set_duration(pc[2].duration)
                          .crossfadeout(0.3))
    
    with_sc = (TextClip('WITH SASHA COBRA', font='Montserrat-SemiBold', fontsize=30, color='white', align='center')
                        .set_position((0.652, 0.55), relative=True)
                        .set_duration(pc[2].duration)
                        .crossfadein(0.3)
                        .crossfadeout(0.3))
    
    watch_fv = (TextClip('Watch full video series', font='Montserrat-SemiBold', 
                        fontsize=40, color='white', align='center')
                        .set_position(('center', 'center'))
                        .set_duration(pc[3].duration)
                        .crossfadein(0.3)
                        .crossfadeout(0.3))
    
    at_sc = (TextClip('at sashacobra.com', font='Montserrat-SemiBold',
                      fontsize=40, color='white', align='center')
                      .set_position(('center', 0.52), relative=True)
                      .set_duration(pc[3].duration)
                      .crossfadein(0.3)
                      .crossfadeout(0.3))
    
    #Defining final logo image clip
    final_logo_4 = (ImageClip("/home/rohit/Videos/sasha_clips/logos/logo_name/logo_name_21.png")
                    .set_duration(pc[4].duration)
                    .set_pos(("center", "center"))
                    .resize(0.65)
                    .crossfadein(0.3))
    
    final_logo_5 = (ImageClip("/home/rohit/Videos/sasha_clips/logos/logo_name/logo_name_21.png")
                    .set_duration(pc[5].duration)
                    .set_pos(("center", "center"))
                    .resize(0.65)
                    .crossfadeout(pc[5].duration))
    
    #Generating composite video clips
    comp1 = CompositeVideoClip([pc[1], getting_intimate_1])
    comp2 = CompositeVideoClip([pc[2], getting_intimate_2, with_sc])
    comp3 = CompositeVideoClip([pc[3], watch_fv, at_sc])
    comp4 = CompositeVideoClip([pc[4], final_logo_4])
    comp5 = CompositeVideoClip([pc[5], final_logo_5])
    
    post_clip_with_text = concatenate_videoclips([comp1, comp2, comp3, comp4, comp5])
    
    #Setting post_clip audio

    #Importing audio file
    post_clip_audio = AudioFileClip('/home/rohit/Videos/sasha_clips/temp/intro_audio.wav')
    
    #Setting audio
    post_clip_final = post_clip_with_text.set_audio(post_clip_audio)
    
    #Audio fade in and out effects
    post_clip_final = post_clip_final.audio_fadein(0.3)
    post_clip_final = post_clip_final.audio_fadeout(pc[5].duration)
    
    #post_clip_final.write_videofile('/home/rohit/Videos/sasha_clips/post_clip_ani_vertical.mp4', logger=None)
    
    return post_clip_final

In [13]:
##Looping through the clips and generate the subclips
#for i in clips.keys():
#    
#    #Extracting clip data
#    title = clips[i]['title']
#    start = clips[i]['start']
#    end = clips[i]['end']
#    
#    #Extracting the subclip from the main file
#    main_clip_original = VideoFileClip(original_video_file).subclip(start, end)
#    
#    #Cropping clip
#    main_clip_crop = crop_clip(main_clip_original)
#    
#    #Saving video
#    print('processing ' + title)
#    #main_clip_crop.write_videofile(temp_folder + title + '.mp4', logger=None)

In [ ]:
#Looping through the clips
for i in clips.keys():
    
    if i <= 21:
        continue
    
    #Extracting the title
    title = clips[i]['title']
    start = clips[i]['start']
    end = clips[i]['end']
   
    print('Processing initiated for ' + title)

    #Defining the clip path
    video_clip_url = temp_folder + title + '.mp4'
    
    #Converting clip to movie py object
    video_clip_original = VideoFileClip(video_clip_url)
    
    #Adding logo
    clip_with_logo = add_logo(video_clip_original) 
    
    #Adding subtitles
    clip_with_subtitles = add_subtitle(clip_with_logo, title)
    
    #Adding post clip animation
    post_clip = add_post_clip_ani(main_clip_end_time=end)
    
    #Putting main clip and post clip together
    final_export = concatenate_videoclips([clip_with_subtitles, post_clip], method='chain')
    
    #Exporting
    export_file_path = generated_clips_directory + title + '.mp4'
    final_export.write_videofile(export_file_path, logger=None)
    
    print('Finished processing ' + title + '.mp4')
    
    if i == 25:
        break

Processing initiated for 22_relax_into_masculine_that_you_are
